In [9]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from unidecode import unidecode

distance=pd.read_excel("Turkish network.xls",sheet_name="Distance (km)", skiprows=[0], index_col=1).iloc[:,1:]
travel_time=pd.read_excel("Turkish network.xls",sheet_name="Travel time (min)", skiprows=[0], index_col=1).iloc[:,1:]
flow = pd.read_excel("Turkish network.xls",sheet_name="Flow", skiprows=[0], index_col=1).iloc[:,1:]
fixed_link_cost = pd.read_excel("Turkish network.xls",sheet_name="Fixed link cost", skiprows=[0], index_col=1).iloc[:,1:]
fixed_hub_cost = pd.read_excel("Turkish network.xls", sheet_name="Fixed hub cost", index_col=1).iloc[:,1:]

cities = list(distance.columns)
cities = [unidecode(city) for city in cities]

distance.index = cities
distance.columns = cities

travel_time.index = cities
travel_time.columns = cities

flow.index = cities
flow.columns = cities

fixed_link_cost.index = cities
fixed_link_cost.columns = cities

fixed_hub_cost.index = cities

In [10]:
distance

,ADANA,ADIYAMAN,AFYON,AGRI,AMASYA,ANKARA,ANTALYA,ARTVIN,AYDIN,BALIKESIR,...,BATMAN,SIRNAK,BARTIN,ARDAHAN,IGDIR,YALOVA,KARABUK,KILIS,OSMANIYE,DUZCE
ADANA,0,329,573,961,613,490,558,1030,894,894,...,614,711,769,1036,1064,893,701,242,85,722
ADIYAMAN,329,0,902,648,636,755,887,755,1223,1223,...,301,478,1038,761,751,1162,970,209,244,991
AFYON,573,902,0,1308,590,256,292,1236,351,321,...,1187,1284,515,1345,1418,336,447,815,658,373
AGRI,961,648,1308,0,735,1052,1429,397,1653,1568,...,369,426,1169,310,143,1360,1099,815,876,1189
AMASYA,613,636,590,735,0,334,874,694,937,833,...,802,984,434,778,845,625,364,668,629,454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YALOVA,893,1162,336,1360,625,407,600,1254,507,216,...,1417,1584,374,1363,1470,0,350,1135,978,171
KARABUK,701,970,447,1099,364,215,735,976,794,558,...,1166,1348,88,1085,1209,350,0,943,786,179
KILIS,242,209,815,815,668,730,800,922,1136,1136,...,468,565,1011,928,918,1135,943,0,157,964
OSMANIYE,85,244,658,876,629,575,643,945,979,979,...,529,626,854,951,979,978,786,157,0,807


In [2]:
u = flow * distance * fixed_link_cost / 1000
inflow = u.sum(axis=1)
inflow = dict(zip(inflow.index, inflow.values))
outflow = u.sum(axis=0)
outflow = dict(zip(outflow.index, outflow.values))

In [3]:
#model
m = gp.Model("q2_part1")
m.ModelSense = GRB.MINIMIZE

#decision variables 
xi=m.addVars(cities,name="hub",vtype=GRB.BINARY)

one_hub=m.addConstr(xi.sum('*') == 1)

m.setObjective(gp.quicksum(xi[i] * (inflow[i]+outflow[i]) for i in cities))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-08


In [4]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 81 columns and 81 nonzeros
Model fingerprint: 0xa9e17c44
Variable types: 0 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+06, 5e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1666660.7651
Presolve removed 1 rows and 81 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1.25501e+06 1.66666e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.255005784623e+06, best bound 1.255005784623e+06, gap 0.0000%
